In [1]:
#importing libraries
import pandas as pd
import pymysql

#establish connection with mysql
myconnection = pymysql.connect(host="127.0.0.1", user="root", passwd="1111", database="project1")

#read the csv file
csv_file_path = r"C:/Users/jagadesh/Documents/Python Scripts/red bus data scraping project/redbus_all_data.csv"
df = pd.read_csv(csv_file_path)

#print the column names to debug
print("columns in DataFrame:", df.columns)

#ensure correct column names and drop unneccessary columns
df = df.rename(columns = lambda x : x.strip()) #remove leading and trailing whitespaces

#rename columns to match expected names
df = df.rename(columns={
    "Bus_name": "Bus_Name",
    "Bus Type": "Bus_Type",
    "Departure Time": "Departure_Time",
    "Arrival Time": "Arrival_Time",
    "Duration": "Duration",
    "Fare": "Fare",
    "Rating": "Rating",
    "Seat_Availability": "Seat_Availability",
    "Bus Link": "Bus_Link",
    "Bus Route": "Bus_Route"
}, inplace = False)

#Handle 'Bus_Type' missing values by filling with "Unknown"
df['Bus_Type'] = df['Bus_Type'].fillna("Unknown")

#Handle 'Departure_Time' and 'Arrival_Time' missing values by filling with "00:00"
df['Departure_Time'] = df['Departure_Time'].fillna("00:00")
df['Arrival_Time'] = df['Arrival_Time'].fillna("00:00")

#Handle 'Duration' missing values by filling with "00:00"
df['Duration'] = df['Duration'].fillna("00:00")

#format the DataFrame columns
try:
    if "Departure_Time" in df.columns:
        df["Departure_Time"] = pd.to_datetime(df["Departure_Time"], format="%H:%M", errors="coerce").dt.time
    if "Arrival_Time" in df.columns:
        df["Arrival_Time"] = pd.to_datetime(df["Arrival_Time"], format="%H:%M", errors="coerce").dt.time

except Exception as e:
    print(f"Error parsing data column: {e}")

# Clean 'Seat_Availability' column and handle NaN values
if 'Seat_Availability' in df.columns:
    df['Seat_Availability'] = df['Seat_Availability'].astype(str).str.extract(r'(\d+)').fillna(0).astype(int)
else:
    print("Warning: 'Seat_Availability' column not found")

#ensure 'Rating' and 'Fare' columns are properly formatted
if "Rating" in df.columns:
    df["Rating"]=pd.to_numeric(df["Rating"], errors="coerce").fillna(0) #convert to float and NaNs with 0
if "Fare" in df.columns:
    df["Fare"]=pd.to_numeric(df["Fare"], errors="coerce").fillna(0) #convert to float and NaNs with 0

#ensure 'Bus_name' column is present and treated as string
if "Bus_Name" in df.columns:
    df["Bus_Name"]=df["Bus_Name"].astype(str).fillna("unknown")
else:
    print("warning: 'Bus_name' column not found")

#map pandas datatypes to MYSQL datatypes
dtype_mapping={
    "object": "TEXT",
    "int64": "INT",
    "int32": "INT",
    "float64": "FLOAT",
    "datetime64[ns]": "DATETIME",
    "datetime.time": "TIME"
}

#replace any spaces with underscore in column names
df.columns = df.columns.str.replace(" ","_")

#generate the formatted string for creating the table with escaped column names and types
column_definitions = [f"`{col}` {dtype_mapping.get(str(df[col].dtype), 'TEXT')}" for col in df.columns]
table_definition = ", ".join(column_definitions)

table_name = "redbus"

#create cursor object
cursor = myconnection.cursor()

#drop table if it exist(optional)
cursor.execute(f"DROP TABLE IF EXISTS {table_name}")

#create table
create_table_query = f"CREATE TABLE {table_name} ({table_definition})"
print("Create Table Query:",create_table_query) #check the query incase of debugging
cursor.execute(create_table_query)

#insert data into the table
insert_query = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({', '.join(['%s'] * len(df.columns))})"

#execute the insert queries
for row in df.itertuples(index=False, name=None):
    cursor.execute(insert_query, row)

#commit the changes
myconnection.commit()

#close the cursor and connection
cursor.close()
myconnection.close()

columns in DataFrame: Index(['Bus_name', 'Bus Type', 'Departure Time', 'Arrival Time', 'Duration',
       'Fare', 'Rating', 'Seat_Availability', 'Bus Link', 'Bus Route'],
      dtype='object')
Create Table Query: CREATE TABLE redbus (`Bus_Name` TEXT, `Bus_Type` TEXT, `Departure_Time` TEXT, `Arrival_Time` TEXT, `Duration` TEXT, `Fare` FLOAT, `Rating` FLOAT, `Seat_Availability` INT, `Bus_Link` TEXT, `Bus_Route` TEXT)


In [ ]:
#print(df.head())  # Display the first few rows



In [ ]:
#print(df.info())  # Display information about the DataFrame, including data types and non-null counts

In [2]:
df

,Bus_Name,Bus_Type,Departure_Time,Arrival_Time,Duration,Fare,Rating,Seat_Availability,Bus_Link,Bus_Route
0,FRESHBUS,Electric A/C Seater (2+2),23:10:00,05:35:00,06h 25m,829.0,4.5,17,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
1,IntrCity SmartBus,Bharat Benz A/C Seater /Sleeper (2+1),23:50:00,05:35:00,05h 45m,579.0,4.4,14,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
2,Zingbus Plus,A/C Seater / Sleeper (2+1),23:38:00,06:05:00,06h 27m,834.0,4.5,30,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
3,AdIntrCity SmartBus,A/C Seater / Sleeper (2+1),23:05:00,06:00:00,06h 55m,839.0,4.4,20,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
4,IntrCity SmartBus,Scania AC Multi Axle Sleeper (2+1),23:59:00,06:30:00,06h 31m,1479.0,4.5,11,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Vijayawada
...,...,...,...,...,...,...,...,...,...,...
1139,Samay Shatabdi Travels Pvt Ltd,AC Sleeper (2+1),22:01:00,06:16:00,08h 15m,1499.0,4.9,1,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Lucknow
1140,Panwar Travels,A/C Sleeper (2+1),17:50:00,01:40:00,07h 50m,1850.0,4.8,26,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Lucknow
1141,Baba Khatushyam Travels Pvt Ltd,A/C Seater / Sleeper (3+1),19:00:00,03:00:00,08h 00m,1050.0,1.5,56,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Lucknow
1142,Travel Point World LLP,A/C Seater / Sleeper (2+2),20:20:00,03:20:00,07h 00m,1999.0,1.6,38,https://www.redbus.in/bus-tickets/gopalganj-to...,Gopalganj (Bihar) to Lucknow
